In [3]:
from ai_model import *

In [4]:
# good case
predict({
    "text":"Here is the entire conversation: name Alex address Sample Street 1 date of birth 13 March 1990 .",
    "task":"getCustomerInfo"
})

INFO:root:retrieveCustomerInfo called with name: Alex, dob: 1990-03-13, address: Sample Street 1.
INFO:root:Attempting to run query: 
    SELECT
        name,
        customer_id as Customer_ID,
        current_product as Current_Product,
        churn_risk as Churn_Risk,
        customer_since as Customer_Since,
        date_of_birth as Date_of_birth,
        address as Address,
        preapproved_for_discount as Preappoved_for_discount
    FROM telco_call_center_ai.customer_info
    WHERE LOWER(name) = LOWER('Alex')
    AND date_of_birth = '1990-03-13'
    AND LOWER(address) = LOWER('Sample Street 1')
    .
Hive Session ID = dc9d4879-879e-4435-93f1-5172ffc23ddb
INFO:root:Customer information retrieved: {'name': 'Alex', 'Customer_ID': 13, 'Current_Product': 'AirSpeed Home', 'Churn_Risk': 1, 'Customer_Since': '2022-01-01', 'Date_of_birth': '1990-03-13', 'Address': 'Sample Street 1', 'Preappoved_for_discount': 'No'}.


{'recommendationText': '{\n  "name": "Alex",\n  "address": "Sample Street 1",\n  "dob": "1990-03-13"\n}',
 'foundCustomer': 1,
 'customerInfo': {'name': 'Alex',
  'Customer_ID': 13,
  'Current_Product': 'AirSpeed Home',
  'Churn_Risk': 1,
  'Customer_Since': '2022-01-01',
  'Date_of_birth': '1990-03-13',
  'Address': 'Sample Street 1',
  'Preappoved_for_discount': 'No'}}

In [5]:
# good case
text = "Here is the entire conversation: name Alex address Sample Street 1 date of birth 13 March 1990 ."

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": 'You are a helpful assistant for call center agents designed to analyze text from a conversation and figure out information about the customer who is calling outputing a JSON. Provide your answer in JSON structure like this {"name": "<The name of the customer>", "address": "<The street of the customer> <The house number of the customer as number, not string>", "dob": "<The date of birth of the customer as YYYY-MM-DD>". You will be given the entire conversation so far, do not worry if the information is not complete yet, just fill out whatever you can identify.'},
        {"role": "user", "content": f"{text}"}
    ]
)

info = json.loads(completion.choices[0].message.content)
info_complete = False
needed_informattion = ['name', 'address', 'dob']

for info_elem in needed_informattion:
    if info_elem in info and info[info_elem] and info[info_elem] != "":
        info_complete = True
        if info_elem == 'dob' and not is_valid_date(info['dob']):
            info_complete = False
    else:
        info_complete = False

info

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'name': 'Alex', 'address': 'Sample Street 1', 'dob': '1990-03-13'}

In [6]:
if info_complete:
    customer_info_from_db = retrieveCustomerInfo(info['name'], info['dob'], info['address'])
    if customer_info_from_db == False:
        output = {"recommendationText": completion.choices[0].message.content,
                  "foundCustomer": 0}
    else:
        output = {"recommendationText": completion.choices[0].message.content,
                  "foundCustomer": 1,
                  "customerInfo": customer_info_from_db}

else:
    output = {
        "recommendationText": completion.choices[0].message.content}

INFO:root:retrieveCustomerInfo called with name: Alex, dob: 1990-03-13, address: Sample Street 1.
INFO:root:Attempting to run query: 
    SELECT
        name,
        customer_id as Customer_ID,
        current_product as Current_Product,
        churn_risk as Churn_Risk,
        customer_since as Customer_Since,
        date_of_birth as Date_of_birth,
        address as Address,
        preapproved_for_discount as Preappoved_for_discount
    FROM telco_call_center_ai.customer_info
    WHERE LOWER(name) = LOWER('Alex')
    AND date_of_birth = '1990-03-13'
    AND LOWER(address) = LOWER('Sample Street 1')
    .
INFO:root:Customer information retrieved: {'name': 'Alex', 'Customer_ID': 13, 'Current_Product': 'AirSpeed Home', 'Churn_Risk': 1, 'Customer_Since': '2022-01-01', 'Date_of_birth': '1990-03-13', 'Address': 'Sample Street 1', 'Preappoved_for_discount': 'No'}.


In [7]:
customer_info_from_db

{'name': 'Alex',
 'Customer_ID': 13,
 'Current_Product': 'AirSpeed Home',
 'Churn_Risk': 1,
 'Customer_Since': '2022-01-01',
 'Date_of_birth': '1990-03-13',
 'Address': 'Sample Street 1',
 'Preappoved_for_discount': 'No'}

In [8]:
# good case
retrieveCustomerInfo("Alex", "1990-03-13", "Sample Street 1")

INFO:root:retrieveCustomerInfo called with name: Alex, dob: 1990-03-13, address: Sample Street 1.
INFO:root:Attempting to run query: 
    SELECT
        name,
        customer_id as Customer_ID,
        current_product as Current_Product,
        churn_risk as Churn_Risk,
        customer_since as Customer_Since,
        date_of_birth as Date_of_birth,
        address as Address,
        preapproved_for_discount as Preappoved_for_discount
    FROM telco_call_center_ai.customer_info
    WHERE LOWER(name) = LOWER('Alex')
    AND date_of_birth = '1990-03-13'
    AND LOWER(address) = LOWER('Sample Street 1')
    .
INFO:root:Customer information retrieved: {'name': 'Alex', 'Customer_ID': 13, 'Current_Product': 'AirSpeed Home', 'Churn_Risk': 1, 'Customer_Since': '2022-01-01', 'Date_of_birth': '1990-03-13', 'Address': 'Sample Street 1', 'Preappoved_for_discount': 'No'}.


{'name': 'Alex',
 'Customer_ID': 13,
 'Current_Product': 'AirSpeed Home',
 'Churn_Risk': 1,
 'Customer_Since': '2022-01-01',
 'Date_of_birth': '1990-03-13',
 'Address': 'Sample Street 1',
 'Preappoved_for_discount': 'No'}

In [9]:
# bad case
retrieveCustomerInfo("Alex", "foo", "Sample Street 1")

INFO:root:retrieveCustomerInfo called with name: Alex, dob: foo, address: Sample Street 1.
INFO:root:Attempting to run query: 
    SELECT
        name,
        customer_id as Customer_ID,
        current_product as Current_Product,
        churn_risk as Churn_Risk,
        customer_since as Customer_Since,
        date_of_birth as Date_of_birth,
        address as Address,
        preapproved_for_discount as Preappoved_for_discount
    FROM telco_call_center_ai.customer_info
    WHERE LOWER(name) = LOWER('Alex')
    AND date_of_birth = 'foo'
    AND LOWER(address) = LOWER('Sample Street 1')
    .
INFO:root:No matching customer found.


False